This is the first notebook to to start fresh for testing the implementation of DC.  
This still assume inverse registration;
checklist for progress, minimal change to run through:  
> (y for done; pr for pending run; x for not implemented yet)
- (y) preview n_raw_image_tiles & tile configuration (n by m)
- (pr) stitching
- (x) destriping
- (pr) downsizing
- (pr) registration
- (pr) upsize atlas
- () make corrected layers
- () make atlas layer
- () add annotations
- () make ng links

The notebook itself should run in any python env that has necessary packages installed. The specific actual commands depends on the conda env `lightsheet` in `pnilsadmin` account, which is an environment that has some changes comparing to the `lightsheet` requirement specified in BrainPipe (both braincogs and PU version). Should be ware of this in case run in other account.

In [ ]:
# import necessary modules
import importlib


### Test DC implementation with one sample;

#### import and intialize the DC

In [58]:
import sys
sys.path.append('/jukebox/wang/yang/_yang_dev/data_organize') # where the code is
#### # additional func to force update in the module without restart the whole kernel
import DataContainer
importlib.reload(DataContainer) 
####
from DataContainer import Datacontainer


sample = '?'

dc = Datacontainer(scope='SS',
               user='sbolkan',
               sample='a2a_230',
               imaging_direction='vd',
               imaging_request=1,
               auto_laser=488,
               atlas_name='allen')
dc.summary()

# primitive form of batching
dc_list = [dc]

===data for user sbolkan, beginning ===/
20241004_12_05_24_sbolkan_a2a_245
20241004_16_47_31_sbolkan_d1_211
20241104_09_40_19_sbolkan_a2a_250
20241104_13_26_03_sbolkan_a2a_492
20241115_12_30_54_sbolkan_a2a_241
20241205_13_29_31_sbolkan-a2a_230-vd-1
===data for user sbolkan, ending ======/
raw channels found: 
Ex_488_Em_525
Ex_561_Em_000
Ex_639_Em_690
/jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1
========= start of summary of class, v0 ============
{
    "version": "v0",
    "scope": "SS",
    "scope_name": "SmartSPIM",
    "user": "sbolkan",
    "sample": "a2a_230",
    "imaging_direction": "vd",
    "imaging_request": "1",
    "raw_dir_root": "/jukebox/LightSheetData/_rawData",
    "process_dir_root": "/jukebox/LightSheetData/_procData",
    "viz_dir_root": "/jukebox/LightSheetData/neuroglancer/public/_ngviz",
    "raw_dir": "/jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1",
    "process_dir": "/juke

#### (additional params setting? skip for now)
> ONE MAY CONSIDER WHICH SHOULD BE SET, WHICH GOES TO HARDCODED

#### preview LS tiles & total files 

In [23]:
import os
from pathlib import Path
for dc in dc_list:
    print(dc.label)
    raw_488_path = dc.raw_channels[488]
    htiles = len([x for x in os.listdir(raw_488_path) if Path(os.path.join(raw_488_path, x)).is_dir()])
    

    raw_488_vtiles_path = [os.path.join(raw_488_path, x) for x in os.listdir(raw_488_path) 
                            if Path(os.path.join(raw_488_path, x)).is_dir()
                            ][0]
    
    vtiles = len([x for x in os.listdir(raw_488_vtiles_path)])
    
    print(str(htiles) + " x " + str(vtiles))

    raw_488_tile_path = [os.path.join(raw_488_vtiles_path, x) for x in os.listdir(raw_488_vtiles_path)][-1]
    num_files = len([x for x in os.listdir(raw_488_tile_path)])
    print(num_files)
    print("")


20241205_13_29_31_sbolkan-a2a_230-vd-1
4 x 5
3283



#### stitching

In [24]:
os.chdir("/jukebox/wang/sanjeev/github_repos/BrainPipe/smartspim/")
print('cd /jukebox/wang/sanjeev/github_repos/BrainPipe/smartspim/')
for dc in dc_list:
    print(dc.label)
    dirs = []
    raw_channel_dirs = dc.raw_channels
    stitched_dirs = dc.get_stitched_dir()

    command_str = "./spim_stitching_pipeline_multichannel.sh"
    for laser in dc.lasers:
        command_str = command_str + f" {raw_channel_dirs[laser]} {stitched_dirs[laser]}"
    
    command = command_str.format(*dirs)

    print(command)

cd /jukebox/wang/sanjeev/github_repos/BrainPipe/smartspim/
20241205_13_29_31_sbolkan-a2a_230-vd-1
./spim_stitching_pipeline_multichannel.sh /jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1/Ex_488_Em_525 /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/488_stitched /jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1/Ex_561_Em_000 /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/561_stitched /jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1/Ex_639_Em_690 /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/639_stitched


#### destriping

In [ ]:
# destriping code

#### downsizing

In [ ]:
os.chdir("/jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/downsizing/")
print('cd /jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/downsizing/')
for dc in dc_list:
    print(dc.label)
    for laser in dc.lasers:
        try:
            downsize_from_dir = dc.get_destriped_dir() # use destriped first (assumption: destripe after stitch; change if order flip)
        except:
            downsize_from_dir = dc.get_stitched_dir()
        downsized_dir =  dc.get_downsized_dir()
        
        command = "sbatch spim_downsize.sh '{}' {} {} {}".format(downsize_from_dir[laser], downsized_dir[laser], dc.imaging_direction[0], dc.atlas_name) # imaging direction compatibility therefore takes the first char.
        #job_output = subprocess.check_output(command, shell=True)
        #job_id = str(job_output)[22:30]
        #print("Running: " + str(sample) + ", channel: " + str(laser))
        #print(job_output)
        print(command)
        print("")

cd /jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/downsizing/
20241205_13_29_31_sbolkan-a2a_230-vd-1
sbatch spim_downsize.sh '/jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/488_destriped' /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/488_downsized v allen

sbatch spim_downsize.sh '/jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/561_destriped' /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/561_downsized v allen

sbatch spim_downsize.sh '/jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/639_destriped' /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/639_downsized v allen



#### Registration (3-chan compatible)

In [54]:
os.chdir("/jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/registration/")
print('cd /jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/registration/')

only_auto_to_atlas = False # temp solution; pending to fix next update
for dc in dc_list:
    print(dc.label)
    # src = LS_dirs.get_rawdata_path(sample)
    src = dc.raw_dir

    print(dc.lasers)
    if only_auto_to_atlas:
        # This skip the pipeline which determine whether it is forward or inverse registration; beware when reuse
        # reg = LS_dirs.get_channel_path(sample, "downsized", lasers[0]) # lasers[0] should be 488
        reg = dc.get_downsized_dir()[dc.auto_laser]
        reg_vol = "downsized_for_atlas.tif"
        cell = "NA" # so that only do registration on that;
        cell_vol = "NA" # placeholder
        command = "./spim_registration_pipeline_semicustom_YL.sh {} {} {} {} {} {} {}".format(src, reg, reg_vol, 
                                                                                            cell, cell_vol, dc.atlas_name, "inverse")
        print(command)
    else: # do registration together with signal channels
        for i in dc.signal_lasers:
            print(f"register {i} to autofluorescence at {dc.auto_laser}")
            # reg = LS_dirs.get_channel_path(sample, "downsized", lasers[0])
            reg = dc.get_downsized_dir()[dc.auto_laser]
            reg_vol = "downsized_for_atlas.tif"
            
            # cell = LS_dirs.get_channel_path(sample, "downsized", lasers[i])
            cell = dc.get_downsized_dir()[i]
            cell_vol = "downsized_for_atlas.tif"
            
            # command = "./spim_registration_pipeline_semicustom.sh {} {} {} {} {} {} {}".format(src, reg, reg_vol, 
            #                                                                                 cell, cell_vol, atl, "inverse")
            ### customized for 3+ channel, forward; one time run for all channel; no repeat run for registration
            command = "./spim_registration_pipeline_semicustom_YL.sh {} {} {} {} {} {} {}".format(src, reg, reg_vol, 
                                                                                            cell, cell_vol, dc.atlas_name, "inverse")
            print(command)
    print("")

cd /jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/registration/
20241205_13_29_31_sbolkan-a2a_230-vd-1
[488, 561, 639]
register 561 to autofluorescence at 488
./spim_registration_pipeline_semicustom_YL.sh /jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1 /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/488_downsized downsized_for_atlas.tif /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/561_downsized downsized_for_atlas.tif allen inverse
register 639 to autofluorescence at 488
./spim_registration_pipeline_semicustom_YL.sh /jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1 /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/488_downsized downsized_for_atlas.tif /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/639_downsized downsized_for_atlas.tif alle

#### (specific in inverse registration) upsize atlas
> Upsize the atlas (that has been mapped to downsized sample space) to raw data resolution

In [ ]:
os.chdir("/jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/raw_space_atlas_parallel/")
print('cd /jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/raw_space_atlas_parallel/')

for dc in dc_list:
    # raw_atlas_dirs = LS_dirs.get_raw_atlas_pipeline_directories(sample, "corrected", lasers[1], "elastix_inverse_transform", 
    #                                                             ra_dirname)
    raw_atlas_dirs = dc.get_raw_atlas_dirs()
    command = "./run_transform_pipeline_sanjeev.sh '{}' {} {} {} {} {}".format(dc.stitched_dir[dc.lasers[0]], raw_atlas_dirs['atlas_to_template'], 
                                                                             raw_atlas_dirs['template_to_signal'], raw_atlas_dirs['output'],
                                                                             dc.atlas_name, dc.imaging_direction[0]) # first arg. only used to check shape of raw images
    print(command)
    print(" ")

cd /jukebox/wang/sanjeev/lightsheet_helper_scripts/smartspim_pipeline_AH/raw_space_atlas_parallel/
./run_transform_pipeline_sanjeev.sh '/jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/488_stitched' /jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/elastix_inverse_transform/atl_to_488 {561: '/jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/elastix_inverse_transform/488_to_561', 639: '/jukebox/LightSheetData/_procData/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1-SS/elastix_inverse_transform/488_to_639'} /jukebox/LightSheetData/_rawData/SmartSPIM/sbolkan/20241205_13_29_31_sbolkan-a2a_230-vd-1/raw_atlas allen v
 


#### make precomputed layers for signal channels

In [ ]:
# need to implement dirs in dc; check the param file for ng pipelines;

os.chdir("/jukebox/wang/sanjeev/github_repos/precomputed/smartspim/corrected_pipeline/")
print('cd /jukebox/wang/sanjeev/github_repos/precomputed/smartspim/corrected_pipeline/')

for sample in samples:
    for laser in lasers:
        viz_dir = os.path.join(LS_dirs.get_viz_path(sample),
                               "ch" + str(laser) + "_corrected")
        res = resolution
        command = "./precomputed_pipeline_corrected.sh " + viz_dir + " " + res + " " + str(laser)
        # job_output = subprocess.check_output(command, shell=True)
 #       print(sample + " " + str(laser))
 #       print(job_output)
        print(command)
        print(" ")

### Test batching code with one sample;
> simplest batch would be dc.; 
> Now, we need to make these blocks into functions (pipeline functions)
> => the main should have a function that pass

### Test batching code with multiple samples;